## Exploration of using Google Places API for use in mapping out science related businesses that have science related job postings in Houston, Texas. 

#### started 2017-02-17

#### import all needed python modules

In [1]:
import requests
import configparser
import json
import geojson
from geojson import Point, Feature, FeatureCollection
# https://github.com/frewsxcv/python-geojson
import time
import pandas as pd

### using <a href="https://docs.python.org/3/library/configparser.html">configparser</a> to handle config file 

In [2]:
config = configparser.ConfigParser()

In [3]:
configFilePath = './config.conf'
# configparser.ConfigParser().read(configFilePath)
config.read(configFilePath)

['./config.conf']

sets key as variable that contains the google-places-api key

In [4]:
# key = configparser['google-places-api']['key1']
key = config['google-places-api']['key1']

In [5]:
key_pre = '&key='

In [6]:
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query='

*next_page_base_url* string used when a result with the base url and query has more than 20 results. Next page includes 20 more results. There is a possiblity for a third list of results, but most of the results with that many locations include many false positives, so only 40 locations were found at max for each search term combination.

In [8]:
next_page_base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'

# Using big list of companies to create single geoJSON

In [9]:
# using pandas as pd. Pandas should already be imported
df=pd.read_csv('../DATA/biglist_2c_resultsAll.csv', sep=',',header=None)
company_array = df.values

len(company_array)

1951

In [10]:
company_array

array([['Doble-Engineering'],
       ['Gulfstream-Services-Inc-1'],
       ['Palm-Health-Resources-1'],
       ..., 
       ['Irhythm'],
       ['Center-For-Toxicology-and-Environmental-Health,-Llc.'],
       ['First-Texas-Hospital---Cy--fair']], dtype=object)

#### reformatting the array to be of strings only

In [11]:
company_array_formatted = []
for each in company_array:
    each = each[0]
    company_array_formatted.append(each)
    
company_array_formatted

['Doble-Engineering',
 'Gulfstream-Services-Inc-1',
 'Palm-Health-Resources-1',
 'Rhino-Marketing,-LLC',
 'Genesis-Energy',
 'Jacobs',
 'Gma-Garnet-Usa%2C-Corp',
 'Rainmaker-Consulting-Resources',
 'The-Rio-Group',
 'Ninyo-&-Moore',
 'Trimac-Transportation',
 'Research-Fumigation-Company-1',
 'Cortland-Partners',
 'Solvay',
 'Catholic-Charities-of-the-Archdiocese-of-Galveston--houston',
 'Ucb',
 'Hilcorp-Energy-Company',
 'Ecmc',
 'Healthcare-Job-Board',
 'The-Goddard-School-of-Lake-Houston',
 'Apex',
 'Brightview',
 'Texas-Tissue-Converting,-LLC',
 'Nestl%C3%A9-Waters-North-America-1',
 'Enerflex',
 'Personal-Touch-Therapy-1',
 'Ctg',
 'Gca-Services-Group',
 'Honeywell',
 'Rollins',
 'City-of-Pasadena',
 'Bny-Mellon',
 'Matrix-Providers',
 'Apria-Healthcare',
 'Dresser--rand',
 'Usi-Insurance-Services',
 'Aquent-9',
 'Infinity-Medstaff',
 'S&B-Infrastructure,-Ltd',
 'Live-Nation',
 'Gulf-Coast-Research-Group',
 'Mckinsey-%26-Company',
 'Thermo-Fisher-Scientific',
 'The-Galloway-School

In [12]:
len(company_array_formatted)

1951

In [13]:
# creating sub-list for quicker testin
company_array_formatted_sublist = ['Doble-Engineering',
 'Gulfstream-Services-Inc-1',
 'Palm-Health-Resources-1',
 'Rhino-Marketing,-LLC',
 'Genesis-Energy']

company_array_formatted_sublist

['Doble-Engineering',
 'Gulfstream-Services-Inc-1',
 'Palm-Health-Resources-1',
 'Rhino-Marketing,-LLC',
 'Genesis-Energy']

### Two functions that create features in an array, that will need to be put in a feature collection for the geoJSON

In [106]:
# function that takes a datset that is 
def makeFeatureCollectionsFromPlaces(search_term,nameOfSearch):
    dataset_I = requests.get(base_url+search_term+key_pre+key)
    next_page_result = []
    try:
        next_page_token = dataset_I.json()['next_page_token']
        next_page_result = callNextPageResults(nameOfSearch,next_page_token)
    except: 
        pass
    #####if 'next_page_token' in dataset_I:
    #next_page_token = dataset_I.json()['next_page_token']
    #print("next_page_token",next_page_token)
    #callNextPageResults(nameOfSearch,next_page_token)
    #####next_page_results = makeFeatureCollectionsFromPlaces(dataset_I['next_page_token'],nameOfSearch).json()['results']
    ##### #else: 
    ####    next_page = "none needed"
    dataset = dataset_I.json()['results']
    #if 'next_page_token' in dataset_I:
    #    dataset.extend(next_page_results)
    #    print("dataset up top",dataset)
    #else:
    #    next_page = "none needed"
    array_of_features = []
    for each in dataset:
        photos = []
        # centerpoint for coordinates
        print(each)
        lat = each['geometry']['location']['lat']
        long = each['geometry']['location']['lng']
        # properties
        address = each['formatted_address']
        id = each['id']
        name = each['name']
        # photos is an array
        try:
            photos = each['photos']
        except:
            photos = []
            #print("photos , attribute error but kept going")
        place_id = each['place_id']
        try:
            rating = each['rating']
        except:
            rating = 'NA'
            #print("rating , attribute error but kept going")
        reference = each['reference']
        # types is an array
        types = each['types']
        testPoint = Point((long, lat))
        test2_geoJSOn = Feature(geometry=testPoint, properties={"name": name,"address":address,"id":id,"photos":photos,"place_id":place_id,"rating":rating,"reference":reference,"types":types})
        array_of_features.extend([test2_geoJSOn])
#     new_FeatureCollection = FeatureCollection(array_of_features) 
#     with open('../DATA/'+nameOfSearch+'_test.geojson', 'w') as f:
#         json.dump(new_FeatureCollection, f)
#     return new_FeatureCollection
    print("array_of_features = ",array_of_features," and next_page_result = ",next_page_result)
    print("types: for "+search_term+" type(next_page_result)= ",type(next_page_result)," and type(array_of_features)=  ",type(array_of_features))
    if next_page_result is None or next_page_result==[]:
        print("no second page")
    else:
        print("next_page_result passed none test and is ",next_page_result)
        array_of_features.extend(next_page_result)
        print("array_of_features after extension = ",array_of_features)
    print("2nd statement of array_of_features = ",array_of_features)
    return array_of_features

In [107]:
def callNextPageResults(nameOfSearch,keyForNextPage):
    time.sleep(5)
    print('next_page_base_url+"&pagetoken="+keyForNextPage+key_pre+key =',next_page_base_url+"pagetoken="+keyForNextPage+key_pre+key)
    dataset_I = requests.get(next_page_base_url+"pagetoken="+keyForNextPage+key_pre+key)
    print("dataset_I second page full response",dataset_I.json())
    dataset = dataset_I.json()['results']
    array_of_features = []
    for each in dataset:
        photos = []
        # centerpoint for coordinates
        print(each)
        lat = each['geometry']['location']['lat']
        long = each['geometry']['location']['lng']
        # properties
        address = each['formatted_address']
        id = each['id']
        name = each['name']
        # photos is an array
        try:
            photos = each['photos']
        except:
            photos = []
            #print("photos , attribute error but kept going")
        place_id = each['place_id']
        try:
            rating = each['rating']
        except:
            rating = 'NA'
            #print("rating , attribute error but kept going")
        reference = each['reference']
        # types is an array
        types = each['types']
        testPoint = Point((long, lat))
        test2_geoJSOn = Feature(geometry=testPoint, properties={"name": name,"address":address,"id":id,"photos":photos,"place_id":place_id,"rating":rating,"reference":reference,"types":types})
        array_of_features.extend([test2_geoJSOn])
    #new_FeatureCollection = FeatureCollection(array_of_features) 
    #with open('../DATA/'+nameOfSearch+'_2ndPage_test.geojson', 'w') as f:
    #    json.dump(new_FeatureCollection, f)
    #print("second page results = ",new_FeatureCollection)
    #return new_FeatureCollection
    return array_of_features

### A function that interates through the array of companies and collects the arrays of geographic features for the geojson.

In [87]:
def createGeoJSONfromList(cp_array_formatted):
    array_of_all_features = []
    for each in cp_array_formatted:
        #try:
        features_1 = makeFeatureCollectionsFromPlaces(each+"+Houston+Texas","bigTest1")
        print("features for a company name =",features_1)
        if array_of_all_features == [] or array_of_all_features is None:
            array_of_all_features = features_1
        else:
            if features_1 is None or features_1 == []:
                pass
            else:
                array_of_all_features.extend(features_1)
        #except:
        #    print("error in except",error)
        
        print("array_of_all_features = ",array_of_all_features)
    
    return array_of_all_features

In [56]:
company_array_formatted_sublist

['Doble-Engineering',
 'Gulfstream-Services-Inc-1',
 'Palm-Health-Resources-1',
 'Rhino-Marketing,-LLC',
 'Genesis-Energy']

In [39]:
new_FeatureCollection = FeatureCollection(createGeoJSONfromList(company_array_formatted_sublist))
new_FeatureCollection

{'formatted_address': '1000 S Loop W #100, Houston, TX 77054, United States', 'reference': 'CmRRAAAAdzUGcA9Da7y8gd6WmH9SpxEcVvGJBz8d2xVPD7w7ApbkvMCgXyznyUeAD1lFBuuAp8DhKPIQQQAq-76D6i5Q84A_ETV9UjanVo1FSDWJSbk0e2nPvJziuZHqzQ6kYZABEhDwWcxlQAdSQIUT1_qhj-W9GhRXiNm-ZOALuxSSfE8yCOMVlME1fA', 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'geometry': {'viewport': {'southwest': {'lng': -95.3918668302915, 'lat': 29.6798140697085}, 'northeast': {'lng': -95.3891688697085, 'lat': 29.68251203029149}}, 'location': {'lng': -95.39051959999999, 'lat': 29.6812243}}, 'name': 'Doble', 'id': 'd7d864991744cf9af94c72de181f4bf267524e01', 'types': ['point_of_interest', 'establishment'], 'place_id': 'ChIJBwBYRv6_QIYRatQETbtlQXU'}
{'formatted_address': '12319 Lynda Dr, Houston, TX 77038, United States', 'types': ['electrician', 'point_of_interest', 'establishment'], 'reference': 'CmRRAAAAec4cvBVkb7PjK1NDvl-zDEsus9M49JSlYjjdDTEvuscfPG6V9JYwmP-sO7xVG62NBJTpg3OuhK7f6_JNhlul3taph0

{"features": [{"geometry": {"coordinates": [-95.39051959999999, 29.6812243], "type": "Point"}, "properties": {"address": "1000 S Loop W #100, Houston, TX 77054, United States", "id": "d7d864991744cf9af94c72de181f4bf267524e01", "name": "Doble", "photos": [], "place_id": "ChIJBwBYRv6_QIYRatQETbtlQXU", "rating": "NA", "reference": "CmRRAAAAdzUGcA9Da7y8gd6WmH9SpxEcVvGJBz8d2xVPD7w7ApbkvMCgXyznyUeAD1lFBuuAp8DhKPIQQQAq-76D6i5Q84A_ETV9UjanVo1FSDWJSbk0e2nPvJziuZHqzQ6kYZABEhDwWcxlQAdSQIUT1_qhj-W9GhRXiNm-ZOALuxSSfE8yCOMVlME1fA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4510504, 29.9099788], "type": "Point"}, "properties": {"address": "12319 Lynda Dr, Houston, TX 77038, United States", "id": "b602b2c55fe6130ab68dbfd31db4382763879b0c", "name": "Doble E Electric", "photos": [{"height": 1224, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/110775628567700969334/photos\">Doble E Electric</a>"], "photo_reference": "CoQ

In [40]:
new_FeatureCollection

{"features": [{"geometry": {"coordinates": [-95.39051959999999, 29.6812243], "type": "Point"}, "properties": {"address": "1000 S Loop W #100, Houston, TX 77054, United States", "id": "d7d864991744cf9af94c72de181f4bf267524e01", "name": "Doble", "photos": [], "place_id": "ChIJBwBYRv6_QIYRatQETbtlQXU", "rating": "NA", "reference": "CmRRAAAAdzUGcA9Da7y8gd6WmH9SpxEcVvGJBz8d2xVPD7w7ApbkvMCgXyznyUeAD1lFBuuAp8DhKPIQQQAq-76D6i5Q84A_ETV9UjanVo1FSDWJSbk0e2nPvJziuZHqzQ6kYZABEhDwWcxlQAdSQIUT1_qhj-W9GhRXiNm-ZOALuxSSfE8yCOMVlME1fA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4510504, 29.9099788], "type": "Point"}, "properties": {"address": "12319 Lynda Dr, Houston, TX 77038, United States", "id": "b602b2c55fe6130ab68dbfd31db4382763879b0c", "name": "Doble E Electric", "photos": [{"height": 1224, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/110775628567700969334/photos\">Doble E Electric</a>"], "photo_reference": "CoQ

In [41]:
# convert feature collection to geoJSON
def convertFeatureCollectionToGeoJSON(featCollection,fileNamePath):
    with open(fileNamePath, 'w') as f:
        json.dump(featCollection, f)


In [42]:
convertFeatureCollectionToGeoJSON(new_FeatureCollection,'../DATA/Big_company_list_v3small_test.geojson')

### Another test to make sure the next page functionality works - this one also includes Shell, which should have a lot of gas stations in Houston.

In [82]:
# creating sub-list for quicker testin
company_array_formatted_sublist2 = ['Genesis-Energy','Shell']

company_array_formatted_sublist2

['Genesis-Energy', 'Shell']

In [108]:
new_FeatureCollection2 = FeatureCollection(createGeoJSONfromList(company_array_formatted_sublist2))
new_FeatureCollection2

{'formatted_address': '919 Milam St #2100, Houston, TX 77002, United States', 'rating': 1, 'reference': 'CmRSAAAATubIDf7z1sl9KG38m25lAsmOkK4yx6tZ0cbbCdzqtJLkydLAi9ALPz-qcADsBFC-dY-OfmLr-Z3NSEt7c-kxQwEEB_Q8B0nIeSvL6eZxX0fciqS8GxtVV8mygYdztAtlEhAbWmQHfn8GOuQV8hzO7hsYGhS1rJHHoMEl8N-PP71ib3FXB6LACQ', 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'geometry': {'viewport': {'southwest': {'lng': -95.36747293029151, 'lat': 29.7568735697085}, 'northeast': {'lng': -95.36477496970849, 'lat': 29.7595715302915}}, 'location': {'lng': -95.3659038, 'lat': 29.75805819999999}}, 'name': 'Genesis Energy, LP', 'id': '7dbf977f8ebe642724c868999336f79ab8361447', 'types': ['point_of_interest', 'establishment'], 'place_id': 'ChIJfy2iBDm_QIYRJ3VGXRbTSYg'}
array_of_features =  [{"geometry": {"coordinates": [-95.3659038, 29.75805819999999], "type": "Point"}, "properties": {"address": "919 Milam St #2100, Houston, TX 77002, United States", "id": "7dbf977f8ebe642724c868999336f79

{"features": [{"geometry": {"coordinates": [-95.3659038, 29.75805819999999], "type": "Point"}, "properties": {"address": "919 Milam St #2100, Houston, TX 77002, United States", "id": "7dbf977f8ebe642724c868999336f79ab8361447", "name": "Genesis Energy, LP", "photos": [], "place_id": "ChIJfy2iBDm_QIYRJ3VGXRbTSYg", "rating": 1, "reference": "CmRSAAAATubIDf7z1sl9KG38m25lAsmOkK4yx6tZ0cbbCdzqtJLkydLAi9ALPz-qcADsBFC-dY-OfmLr-Z3NSEt7c-kxQwEEB_Q8B0nIeSvL6eZxX0fciqS8GxtVV8mygYdztAtlEhAbWmQHfn8GOuQV8hzO7hsYGhS1rJHHoMEl8N-PP71ib3FXB6LACQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.372894, 29.747668], "type": "Point"}, "properties": {"address": "2220 Main St, Houston, TX 77002, United States", "id": "a008b524f02cd1beaed229f80d06f23a73b68bb7", "name": "Shell", "photos": [{"height": 2183, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/105976331058990070645/photos\">WOLFGANG DEMINO</a>"], "photo_reference": "CoQBdwAAA

In [109]:
convertFeatureCollectionToGeoJSON(new_FeatureCollection2,'../DATA/Big_company_list_v4small_PlusShell_test.geojson')

In [ ]:
convertFeatureCollectionToGeoJSON(new_FeatureCollection,'../DATA/Big_company_list_v3small_test.geojson')

In [96]:
testArray4 = [{"geometry": {"coordinates": [-95.412966, 29.706454], "type": "Point"}, "properties": {"address": "2302 W Holcombe Blvd, Houston, TX 77030, United States", "id": "126915b3e3c7cc252e8aacbb459f3805230741b3", "name": "Shell", "photos": [], "place_id": "ChIJMw9zOmrAQIYRkpd4wCH_QlM", "rating": 3.5, "reference": "CmRRAAAAHWcQxO3UYvnEPP_LpHTHN09gPwRwZuS78sypQY-HxSwqmcIlKjExzrHiKTTGb9ARAl1aQF9QKSgRdHf1FkKjjhmgiXIZgT6YZOdB4ZUcDrOD30yaHXyFidqx0ab2ycbTEhAzbEAg3CmeN7v2eZ-RVKtnGhSWBxHKgily1adbesV1S4vEV3zisg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.294772, 29.702266], "type": "Point"}, "properties": {"address": "7001 Gulf Fwy, Houston, TX 77087, United States", "id": "9364f2933512ea2c8c79073edf93948b93c6eb0a", "name": "Shell", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/108597419151098384564/photos\">Joshua Suaris</a>"], "photo_reference": "CoQBdwAAALpSYo2zEozfZpR00JK5g6IxfQ8n1len3YHePGW4YLzVjin7zxSclmTvLzSfbZMODek_SDDb6ljyo9QUh6Ktpbj84lE-7co7qc7QQ-B3_-lnjXgqJycHVaBFcmW8-b97VKxMgq3SQhNco-0dpVY3XYad7bcj7GtBpSA0e810_Lb0EhB2iBf0xVdQTvDpG1oi3bTeGhROIzSluQvxT-qJD0VIDXn5AxPwgQ", "width": 5312}], "place_id": "ChIJK62tYem9QIYRU3EibjOCzDY", "rating": 3.7, "reference": "CmRRAAAAywzxbRKXEjc1CKa4KMAy5bUGgwM_3JWGl-KSyokkCdpliSNrveV-Zz1OWqnDXFc_pcASzReB7do-kc3pUicYrfUlfNevnTJlVzcBvClmSh5gEsdhmNlmXAfDJPdCW4VPEhBJhuHoHXZlo6jrBw4t3Kt8GhRgUp4vYfTVbQgzTiFJMs_0Om8-Sw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.556309, 29.689642], "type": "Point"}, "properties": {"address": "8181 W Sam Houston Pkwy S, Houston, TX 77072, United States", "id": "3cdcc717b958528810c6c003ce772abdc2587bb4", "name": "Shell", "photos": [{"height": 1836, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/111840420575909736094/photos\">Luis M Chen</a>"], "photo_reference": "CoQBdwAAAGJUtbc69v2LKbGHuXz3QWQRpCYf6aRhCI03x9mS38TEc70A0AEoVrEGF7YF2ovVltFQiIXs0JJMA1XvR9fKBhoULHAUB8JqtcHrwUSa6CH_1DcOWhGuGOmOge27Ff4vFvZYkkvJ7lEJMb_RvPEywiKbOefgyZRpefY7H_VBRKTaEhC_nkWW4vYMObmc1I0_FO36GhRWM0ZtP8n9CEu700d4NVhOJZP-SQ", "width": 1836}], "place_id": "ChIJqeeHVrHCQIYRkbbZI9cq_DQ", "rating": 3.5, "reference": "CmRRAAAA0R-nmD43rP_Iaa2gDvJX5Aq_8Z5SlxFYfHiOqKDlbpcbJRigyWxJxEJZskC_3FRp6nJYOP3UIvOj3Gth68Os7SwPI3d43ras2muXZZoMr9H_5utqx3s4WFYCzzT0d3h6EhAHDdnnM1V2hMuxpXRDtFSbGhRRVWA9mwHjiuZvJWXeUOr5CVDpVQ", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.349156, 29.760843], "type": "Point"}, "properties": {"address": "2002 Runnels St, Houston, TX 77003, United States", "id": "0975ae5f27cb2ac3ce244f0d0a8944fd76fd73f1", "name": "Shell", "photos": [{"height": 2448, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/102975406394138540343/photos\">Kristy Cooper</a>"], "photo_reference": "CoQBdwAAAKEDle0L7Y1N1C99oL_IHfPP7btH03ddmaKpGHk5rvWLTgA_6zTiChPu3W8tl0H97cXnPj8D-qIsRCZj1KolV7BXBt_JAICHqVPNxW22df7xd2CWzneLvaG1au52Erj4tKLnoOM1dRSiqyWXxVs1QaXMPjbZB-1wnlh_NVGFTaOOEhB9VUSeGwzHdXfVOT3SQk4MGhQrEEZV_-Nk_9fcnbj8BQww_W9qAQ", "width": 3264}], "place_id": "ChIJX4G_ydm-QIYRliMrxDeDzlk", "rating": 3.1, "reference": "CmRRAAAA3zbsmy_IBJWlaQz1kOkRHy4Z-HDFseDB1kLaTZeoMB1jUeLnl7yqvWM-dw0tf6gXNSsklmYjPT9A4cxTvxJJ2F9J77SkEXURhPw9oNzATkjs9RxkFe6K4f09c62gKp61EhB4hXJvk6DtGmD4Fkk5F9yyGhRXWbAiwEt9FiaNd7n3X1bCR7fuiw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.24251699999999, 29.641132], "type": "Point"}, "properties": {"address": "10521 Gulf Fwy, Houston, TX 77034, United States", "id": "b251941efce2b40b212821df7daa7bac2b04dafd", "name": "Shell", "photos": [], "place_id": "ChIJ2xCo2J2XQIYR6BYBFIbu_Xk", "rating": 3.3, "reference": "CmRRAAAAHyULUts2ohuvA762uFeJnBVIQAbPJaxkTBGStDwwCPS6Zy_sxHcIQ5-IfmNYIbGLU1ss7ZMpSMkhKQUggbwsgHUus7qXnOkGZ1IydI_T_DYYLhnlndFBUS-JkXDClBt5EhCQJzxuC1hoXBzoJjzvmviVGhTTfDrpRKiOnkjPYabc8_AacbG0qw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.41036199999999, 29.93139499999999], "type": "Point"}, "properties": {"address": "10 Aldine Bender Rd, Houston, TX 77060, United States", "id": "d6252b306c5ecc238c88552f3d2fdad97738acc8", "name": "Shell", "photos": [{"height": 5312, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/108826302547166423632/photos\">Donnie Roberson</a>"], "photo_reference": "CoQBdwAAAJ4vx-ujIai4jbpMktYY6iZc7Afk6hOIpqROAPUwYFc-Ka6Wts208obBMfNoM6uWQrzomULMajrVkkpP8teZDDXzLvo25nwAXXPRDO7dPZLBLtsMuQzARAcnkv4KD0CPTPHStUOQtGdMi_mJ67_F0NMQsS3JCcAWzSdMf0ysONWcEhDivDy94MmT3mVk0jpEuYeqGhSUeU_U1Mxjq_sSuEuRU9MCfqwdbg", "width": 2988}], "place_id": "ChIJQyJTzMHJQIYRdaAh1yD8jUE", "rating": 5, "reference": "CmRRAAAAlYx2V2Vk8PiYY8AMbCGspvV1iq0TcsZADqCTNOw0rinJITyFdmZQFcVdusjfNXP_315ntdxfgxYU2kRuOAqDESL6Y_ISXNyUsiQFeuZRMSlNhAc2wbZNXef-oy_F6d-mEhAoA94PoWRogwy8unrbs8rYGhQuHm0VyzHy1RrafryeE1mRjyp_YQ", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.386546, 29.698519], "type": "Point"}, "properties": {"address": "2802 Old Spanish Trail, Houston, TX 77054, United States", "id": "c05504d2afbd717238805b77b8877c0115038f11", "name": "Shell", "photos": [], "place_id": "ChIJlYiQtfO_QIYRpAedOekRI1w", "rating": 3.4, "reference": "CmRRAAAAp79hd75mvvNt4STlgrPcRaQCfmou5FFg3Hw4gCMECr-dDAslL82ECUDr0wgHAzgTzVFdAXpp5BKZbIn-TIORhyvvFe18GrWY8cwDFSkmY_QeUtDcFxdswq2prKxWIxHYEhAR9tNEPKWofNnQZDQa8RktGhSCCzdoDtxLxQHp5L9k3KUySDQhkg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.645299, 29.78831], "type": "Point"}, "properties": {"address": "1219 Highway 6 N, Houston, TX 77084, United States", "id": "12c62a970853db13a22ba2a44af53880c0d67df7", "name": "Shell", "photos": [{"height": 4032, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/108845175708315403750/photos\">Chris Desrochers</a>"], "photo_reference": "CoQBdwAAAA7qou5dq0ijIHXDupd6JbboKK4j62VsvL4SvH1UVoH-t12yoF8VaCrIBqEnXDDgBjS5TuWw5MEfdp317n2-E9eMKePQ2qvLBenc4Uqs4ZW4ysu-Ep8JuS6zcipm0Shp_m73djb8wrl5FRplBreGrVhSkR466_8dCmVyvOK_N-AwEhAHlKlinFZXGZshphzpLzrQGhS5xpJj7pI7g0qx2P0f_INaZM5Yew", "width": 3024}], "place_id": "ChIJOSiyQt_bQIYR2TQNbN8-4CA", "rating": 2.8, "reference": "CmRRAAAArNTp42kyQHUmzLGQM4-moPG-5PTkcnRbyNN2faqJx9CK_navXjw0rjGv2wqCWj4Vwe7SaEurvuojtl2VQoLuN4WtsLaS7wGv6Jfxmd9DrEQrfxN-8X2Qcemjq_kwUi06EhBmtuL7OBSFSlXmYjMTbrtEGhQeJnQqF4g3MVfRIXMPoBBaQvMf4w", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.346826, 29.735939], "type": "Point"}, "properties": {"address": "3505 Gulf Fwy, Houston, TX 77003, United States", "id": "a5933680c7c3fb0c3af444460fe49108e2f074d3", "name": "Shell", "photos": [], "place_id": "ChIJh8f2gPu-QIYRUXOjjflieG4", "rating": 3.5, "reference": "CmRRAAAA65dE4m2qveuQpHl4uM-78p8vTRbgl00zcuauUfK_TuWM6Wne7Z6FXuoN9fw_YbHr3Zgdv9JqGcxi6SWFQKXkH4iVnK2K7n2hr9TN0-u6PxQYp_ExQSck9gmbKU1aSTWiEhAMNWHUyyyOQHmjZgEagVRKGhTKuEXkDWlaijOHG0RHM7311knayw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.316063, 29.77356899999999], "type": "Point"}, "properties": {"address": "5440 I 10 East, Houston, TX 77020, United States", "id": "f22ce12f26ce5067a9befd2c0364a0d8e6081f94", "name": "Shell", "photos": [], "place_id": "ChIJdYl3oay-QIYRjMrgfHiuYeI", "rating": 4.3, "reference": "CmRSAAAAeL988M76g10aTx2rr7v_DdPVDM1eGrFHI45beL_uRVugRRQ1o1l1-vFrNeXCsGL2TvXbUjFKmWt7zaBTeIEpCa_z_G_CZudSygH6rpKM6W0B1kL4lcpA_IUjAW4XqNWREhBmYLUEhhyUH--3KI-WtufPGhT-VuRN0dXfOii6FZrOC7rNTerVbg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.18282320000002, 29.8058657], "type": "Point"}, "properties": {"address": "14760 Wallisville Rd #77049, Houston, TX 77049, United States", "id": "571890c337968a73b1da0fa79049dcd33e9b1826", "name": "Shell Gas Station", "photos": [], "place_id": "ChIJj7TLrl-kQIYRK-1PTcVreUg", "rating": "NA", "reference": "CmRRAAAAPcobSXtLFwYCpcVUmtdOt_Bq4Dw2ogz2KMPmUr0D17Zqw1OsJA8xHlmYIVCc9njVwuOM0iHDGkhN4P-mgGvqp9nbLLnNc6ZPydW-VNJoBQEcQoPu0yfJUftUktM8mXbVEhBUenZbrxVPORNAZwNk090qGhSddAadkixRvozXc9xAhK1iBXCYIw", "types": ["gas_station", "convenience_store", "food", "store", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3654938, 29.7602374], "type": "Point"}, "properties": {"address": "700 Milam St # 125, Houston, TX 77002, United States", "id": "15e18e98bcb38bed8a1e0e52a360f52e49ede007", "name": "Shell Oil Co", "photos": [], "place_id": "ChIJ-yHQKDq_QIYRELnaVCFbiQk", "rating": 3, "reference": "CmRRAAAA3oW6zCyJNy_5E1VgdXfnersqWHdpVUwODk72ih9KJ_y66c-ZYGuoPuPYtNZKQLfLozWd8sK3hkx0giVUVDCThI10WiKr4VBmAfqQa0ClH-JB7_6ND-dy13Y-hDEoFDGXEhCnK6DtKaOUfVU2p_gsG_XyGhSwh-43JulbPXvJcB18187UsBGYtA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.64317199999999, 29.690113], "type": "Point"}, "properties": {"address": "8335 S Texas 6, Houston, TX 77083, United States", "id": "30883d5d1f6bd43eab59d5f5db6ab64f23985a75", "name": "Shell", "photos": [], "place_id": "ChIJzYLoWuDdQIYRMYkyJM-lPx4", "rating": 4.2, "reference": "CmRRAAAACGlaUXr5Wtmc9tcnILYVMEG5IL6muujMH7k9a_B1DaiVVKReOHpsPOJ74TKzNJsIDq6LTubbzMv5smswiYXIDhEFyMd-Lxntqerd2Z-Vyk9SOCuTEKBpLbJEhLiZNmcdEhAkhUf_1rEfmGNCJrcXuARtGhQzb3W9Ze9_05ePcg9J3kO54UUJOw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.36766109999999, 29.7591382], "type": "Point"}, "properties": {"address": "910 Louisiana St, Houston, TX 77002, United States", "id": "0021352724df52d54019ec1367c3add6a9213250", "name": "Shell Chemicals", "photos": [], "place_id": "ChIJ-yHQKDq_QIYRpKwUxYNf1pc", "rating": 5, "reference": "CmRSAAAAKV9C-uu5XTuLFYBrDdiZsK3A0mG83onPAz9SlrgP-kU0T8do5K6sUjriieSZB2E7YsL-u2CYcGiQFV7dB34nuRKVrClRv3PaJMeC6xmUpwXnV_VTAgXrSMuf12FfhzSmEhCyGN6nvw9LQknwZ-XITnZoGhTSn8HzFV07Qf2MOdSA8gcg5CKfww", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.375708, 29.73222500000001], "type": "Point"}, "properties": {"address": "4102 Almeda RD, Houston, TX 77004, United States", "id": "38c0f74426d16401510dac6a7c11f42c0d361879", "name": "Shell", "photos": [], "place_id": "ChIJm-tuf3O_QIYRqOeliKHn5YI", "rating": 2.7, "reference": "CmRSAAAAKLYOvrewesNh5JxB7LqELZ1UUHcvJmpxgpmFJAqnxh1A3gDn3SzCYIHDugRa_W8WMr4_wX5dq3W_zJgy45JQc1Ttv4QzkpbNyJNRkJhKHnZ7TmCm1x0WOlcBW9fxJGhzEhBqkknNX9sSUhjHM_ORXHcIGhS5gRrS_6OMpP_ohTTE0cBN09SgDw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.36566099999999, 29.75684600000001], "type": "Point"}, "properties": {"address": "1000 Main St, Houston, TX 77002, United States", "id": "136bd91368a7903337a6ba59eced1feddaa5641e", "name": "Shell", "photos": [], "place_id": "ChIJgyHmO-HnQIYRqIg0YBzy5ZU", "rating": 4.8, "reference": "CmRSAAAAxw0olN7Pv14Xkj3YLMHMhocoEX1Ds0Vd3Uc3aJCYKfyFCFY24zFC8wgveRl0cSagWVhmdxKQGKW8T1MBLwd2kNBMW-dvxgunQin1bdeet85IgaUvygkGaRA1X1H1rIZeEhBEa0YBQ9ItXhXZVlecC7H9GhQn9UaI8kGCxzWiJwpM-byvUIFhMA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.33626199999999, 29.658344], "type": "Point"}, "properties": {"address": "8601 Martin Luther King Blvd, Houston, TX 77033, United States", "id": "63ab1e40f39f4a4f9443666bcbe77c2f8f46f6d4", "name": "Shell", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/116431771714044812578/photos\">rf oduol</a>"], "photo_reference": "CoQBdwAAACJypp36cjYEPnOcinRrwhszZuNBedE86cpqbG9Tx9Rp0xrHsP45EZafU2xU2JT0wvyciGbTTi0fzl7PhBm1Sh6TJI_init-7XyWoCjQO6YeWq2EM7G45bOowxkpIJhBTuNFu6PAareSmqzG8tDCFfo26Kzg_yEUcvjghp3TGLXOEhDe7cd-0F_VKrRv8i7r-q0GGhROt4UXHo4129VqrSkOz3qM5ZPg9w", "width": 5312}], "place_id": "ChIJlfRGAb6VQIYRo6gggXoJcn8", "rating": 3.9, "reference": "CmRRAAAAv7_GTj4wi6cMbz-I_cixFArNW1ZVN3Rx84j9bIXkmnBiCgrZFO0F_2ltF8IwlKbBBN7JrW4GQwfsHEirpWQcQ8g72UOEG0VUATyPZ7PwqxzEzP1LpZ_YpaaFi60HYDHyEhCbgmaoNAAVidJj9yVI1KcLGhS_PBoNCqxdd1a9V17IVp33q1wQog", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.625976, 29.759544], "type": "Point"}, "properties": {"address": "1402 Eldridge Pkwy, Houston, TX 77077, United States", "id": "534f72a95f674efa318c75ebf7fdfe31019003b5", "name": "Shell", "photos": [], "place_id": "ChIJxfpp3grcQIYRxKQZj-99VA4", "rating": 3.6, "reference": "CmRRAAAAQqAm8f8R7hwgk-3l0DWigtv4dvh4oi35kDG_LHRiucME-YohigWLLOtJ4iYtuaUu2PirLjyLAslTQKvfrjUkyWxl54T---_8Ur75XpYu2rfFYaZMywuIUPzqRY3TlKlyEhBcS78sQgLGI4CpawCCWDYcGhQX-ijbDmxpq0_peZ4WLZJwzNuuTg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.634849, 29.902093], "type": "Point"}, "properties": {"address": "8497 Hwy 6 N, Houston, TX 77095, United States", "id": "9f5fa34f625149ee1765051a13d43a51fded8637", "name": "Shell", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/104840624709152271038/photos\">Trenton Telge</a>"], "photo_reference": "CoQBdwAAAK4w6oR2SEKgk4eguX23RgvS2Cn1o4KUndWgzp9WWn0dQZB96be0USN81_hMd9vbMIs9ZO2QWvd93GbmwvoA5tF9rIsMAEw9_ah-b_xwwbhTQAkg3BI7KubgH6B_eyoQE66dvG3WZHaUEUU9xYXAsjvwzy2ku_Q6Cyl2pbZsLgjREhBX3ZlX2j87kOd-StHD8FdnGhQhXSeTBxK00T9p-5A9yxfce_bXbQ", "width": 5312}], "place_id": "ChIJMarO2yfRQIYRvweAd7ah6lk", "rating": 3.8, "reference": "CmRRAAAAE-fZh89F3yWQULC5OspL7V5jO9fWu08cbyqOn6aCDRu9cl8VKxo7jMnhOrCr5EuaaKMPjgtGBdHsMQ5L6hjjZWLBRNg3iL1w5fDkZxPrTqvzps1YLqo4_Zyr-BJcObyhEhAkZLBcXKWZxlfwNWDa57LgGhTDZDwxP1Uq_7rfdBXVDyESNlgylw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.565243, 29.653341], "type": "Point"}, "properties": {"address": "11503 Southwest Fwy, Houston, TX 77031, United States", "id": "eafc8e7fbe915b243dc9b151d1cc748fa092017e", "name": "Shell", "photos": [{"height": 1836, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/107934174504177841943/photos\">Jari Fuller</a>"], "photo_reference": "CoQBdwAAAARKajk0v359zfInTjgODi1ObhdqUCzYe-FmiC9fjyRkHQoZrQVz-M5jco7EHdnZkODeLGqZCKL4dQ0fuQnfpnaRbqknxZZCvr75y9lP1Pkap_i1lXisO3YQO1n9GqMNxRsVQdHpPlolrwfsGopmnxNrEve24MkpMOWz529lzuPUEhBSmX82MXKSIRsaLhAoCsucGhQ2K7hz9o4jZTpItYlZWo7MK-k_nw", "width": 3264}], "place_id": "ChIJcfq8E-HnQIYRvQ4ZJX9fTgA", "rating": 3.5, "reference": "ClRQAAAAdlzGoBuYGj-ZimumnH8zF4_OFeK_KAeH32iEqBOwJWDZvWr7oSEiEuYkpoIYYxjHey5LARzoE0DCR4sI3AwHtfqJ1nstN6Vdazdf-HxGhZ8SEIo8QJWXdAxLmxsnTkRq4JsaFPct1gT_0NgZywVVWT91EX2NYd88", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}]

In [97]:
testArray5 = [{"geometry": {"coordinates": [-95.412966, 29.706454], "type": "Point"}, "properties": {"address": "2302 W Holcombe Blvd, Houston, TX 77030, United States", "id": "126915b3e3c7cc252e8aacbb459f3805230741b3", "name": "Shell", "photos": [], "place_id": "ChIJMw9zOmrAQIYRkpd4wCH_QlM", "rating": 3.5, "reference": "CmRRAAAAHWcQxO3UYvnEPP_LpHTHN09gPwRwZuS78sypQY-HxSwqmcIlKjExzrHiKTTGb9ARAl1aQF9QKSgRdHf1FkKjjhmgiXIZgT6YZOdB4ZUcDrOD30yaHXyFidqx0ab2ycbTEhAzbEAg3CmeN7v2eZ-RVKtnGhSWBxHKgily1adbesV1S4vEV3zisg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.294772, 29.702266], "type": "Point"}, "properties": {"address": "7001 Gulf Fwy, Houston, TX 77087, United States", "id": "9364f2933512ea2c8c79073edf93948b93c6eb0a", "name": "Shell", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/108597419151098384564/photos\">Joshua Suaris</a>"], "photo_reference": "CoQBdwAAALpSYo2zEozfZpR00JK5g6IxfQ8n1len3YHePGW4YLzVjin7zxSclmTvLzSfbZMODek_SDDb6ljyo9QUh6Ktpbj84lE-7co7qc7QQ-B3_-lnjXgqJycHVaBFcmW8-b97VKxMgq3SQhNco-0dpVY3XYad7bcj7GtBpSA0e810_Lb0EhB2iBf0xVdQTvDpG1oi3bTeGhROIzSluQvxT-qJD0VIDXn5AxPwgQ", "width": 5312}], "place_id": "ChIJK62tYem9QIYRU3EibjOCzDY", "rating": 3.7, "reference": "CmRRAAAAywzxbRKXEjc1CKa4KMAy5bUGgwM_3JWGl-KSyokkCdpliSNrveV-Zz1OWqnDXFc_pcASzReB7do-kc3pUicYrfUlfNevnTJlVzcBvClmSh5gEsdhmNlmXAfDJPdCW4VPEhBJhuHoHXZlo6jrBw4t3Kt8GhRgUp4vYfTVbQgzTiFJMs_0Om8-Sw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.556309, 29.689642], "type": "Point"}, "properties": {"address": "8181 W Sam Houston Pkwy S, Houston, TX 77072, United States", "id": "3cdcc717b958528810c6c003ce772abdc2587bb4", "name": "Shell", "photos": [{"height": 1836, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/111840420575909736094/photos\">Luis M Chen</a>"], "photo_reference": "CoQBdwAAAGJUtbc69v2LKbGHuXz3QWQRpCYf6aRhCI03x9mS38TEc70A0AEoVrEGF7YF2ovVltFQiIXs0JJMA1XvR9fKBhoULHAUB8JqtcHrwUSa6CH_1DcOWhGuGOmOge27Ff4vFvZYkkvJ7lEJMb_RvPEywiKbOefgyZRpefY7H_VBRKTaEhC_nkWW4vYMObmc1I0_FO36GhRWM0ZtP8n9CEu700d4NVhOJZP-SQ", "width": 1836}], "place_id": "ChIJqeeHVrHCQIYRkbbZI9cq_DQ", "rating": 3.5, "reference": "CmRRAAAA0R-nmD43rP_Iaa2gDvJX5Aq_8Z5SlxFYfHiOqKDlbpcbJRigyWxJxEJZskC_3FRp6nJYOP3UIvOj3Gth68Os7SwPI3d43ras2muXZZoMr9H_5utqx3s4WFYCzzT0d3h6EhAHDdnnM1V2hMuxpXRDtFSbGhRRVWA9mwHjiuZvJWXeUOr5CVDpVQ", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.349156, 29.760843], "type": "Point"}, "properties": {"address": "2002 Runnels St, Houston, TX 77003, United States", "id": "0975ae5f27cb2ac3ce244f0d0a8944fd76fd73f1", "name": "Shell", "photos": [{"height": 2448, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/102975406394138540343/photos\">Kristy Cooper</a>"], "photo_reference": "CoQBdwAAAKEDle0L7Y1N1C99oL_IHfPP7btH03ddmaKpGHk5rvWLTgA_6zTiChPu3W8tl0H97cXnPj8D-qIsRCZj1KolV7BXBt_JAICHqVPNxW22df7xd2CWzneLvaG1au52Erj4tKLnoOM1dRSiqyWXxVs1QaXMPjbZB-1wnlh_NVGFTaOOEhB9VUSeGwzHdXfVOT3SQk4MGhQrEEZV_-Nk_9fcnbj8BQww_W9qAQ", "width": 3264}], "place_id": "ChIJX4G_ydm-QIYRliMrxDeDzlk", "rating": 3.1, "reference": "CmRRAAAA3zbsmy_IBJWlaQz1kOkRHy4Z-HDFseDB1kLaTZeoMB1jUeLnl7yqvWM-dw0tf6gXNSsklmYjPT9A4cxTvxJJ2F9J77SkEXURhPw9oNzATkjs9RxkFe6K4f09c62gKp61EhB4hXJvk6DtGmD4Fkk5F9yyGhRXWbAiwEt9FiaNd7n3X1bCR7fuiw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.24251699999999, 29.641132], "type": "Point"}, "properties": {"address": "10521 Gulf Fwy, Houston, TX 77034, United States", "id": "b251941efce2b40b212821df7daa7bac2b04dafd", "name": "Shell", "photos": [], "place_id": "ChIJ2xCo2J2XQIYR6BYBFIbu_Xk", "rating": 3.3, "reference": "CmRRAAAAHyULUts2ohuvA762uFeJnBVIQAbPJaxkTBGStDwwCPS6Zy_sxHcIQ5-IfmNYIbGLU1ss7ZMpSMkhKQUggbwsgHUus7qXnOkGZ1IydI_T_DYYLhnlndFBUS-JkXDClBt5EhCQJzxuC1hoXBzoJjzvmviVGhTTfDrpRKiOnkjPYabc8_AacbG0qw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.41036199999999, 29.93139499999999], "type": "Point"}, "properties": {"address": "10 Aldine Bender Rd, Houston, TX 77060, United States", "id": "d6252b306c5ecc238c88552f3d2fdad97738acc8", "name": "Shell", "photos": [{"height": 5312, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/108826302547166423632/photos\">Donnie Roberson</a>"], "photo_reference": "CoQBdwAAAJ4vx-ujIai4jbpMktYY6iZc7Afk6hOIpqROAPUwYFc-Ka6Wts208obBMfNoM6uWQrzomULMajrVkkpP8teZDDXzLvo25nwAXXPRDO7dPZLBLtsMuQzARAcnkv4KD0CPTPHStUOQtGdMi_mJ67_F0NMQsS3JCcAWzSdMf0ysONWcEhDivDy94MmT3mVk0jpEuYeqGhSUeU_U1Mxjq_sSuEuRU9MCfqwdbg", "width": 2988}], "place_id": "ChIJQyJTzMHJQIYRdaAh1yD8jUE", "rating": 5, "reference": "CmRRAAAAlYx2V2Vk8PiYY8AMbCGspvV1iq0TcsZADqCTNOw0rinJITyFdmZQFcVdusjfNXP_315ntdxfgxYU2kRuOAqDESL6Y_ISXNyUsiQFeuZRMSlNhAc2wbZNXef-oy_F6d-mEhAoA94PoWRogwy8unrbs8rYGhQuHm0VyzHy1RrafryeE1mRjyp_YQ", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.386546, 29.698519], "type": "Point"}, "properties": {"address": "2802 Old Spanish Trail, Houston, TX 77054, United States", "id": "c05504d2afbd717238805b77b8877c0115038f11", "name": "Shell", "photos": [], "place_id": "ChIJlYiQtfO_QIYRpAedOekRI1w", "rating": 3.4, "reference": "CmRRAAAAp79hd75mvvNt4STlgrPcRaQCfmou5FFg3Hw4gCMECr-dDAslL82ECUDr0wgHAzgTzVFdAXpp5BKZbIn-TIORhyvvFe18GrWY8cwDFSkmY_QeUtDcFxdswq2prKxWIxHYEhAR9tNEPKWofNnQZDQa8RktGhSCCzdoDtxLxQHp5L9k3KUySDQhkg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.645299, 29.78831], "type": "Point"}, "properties": {"address": "1219 Highway 6 N, Houston, TX 77084, United States", "id": "12c62a970853db13a22ba2a44af53880c0d67df7", "name": "Shell", "photos": [{"height": 4032, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/108845175708315403750/photos\">Chris Desrochers</a>"], "photo_reference": "CoQBdwAAAA7qou5dq0ijIHXDupd6JbboKK4j62VsvL4SvH1UVoH-t12yoF8VaCrIBqEnXDDgBjS5TuWw5MEfdp317n2-E9eMKePQ2qvLBenc4Uqs4ZW4ysu-Ep8JuS6zcipm0Shp_m73djb8wrl5FRplBreGrVhSkR466_8dCmVyvOK_N-AwEhAHlKlinFZXGZshphzpLzrQGhS5xpJj7pI7g0qx2P0f_INaZM5Yew", "width": 3024}], "place_id": "ChIJOSiyQt_bQIYR2TQNbN8-4CA", "rating": 2.8, "reference": "CmRRAAAArNTp42kyQHUmzLGQM4-moPG-5PTkcnRbyNN2faqJx9CK_navXjw0rjGv2wqCWj4Vwe7SaEurvuojtl2VQoLuN4WtsLaS7wGv6Jfxmd9DrEQrfxN-8X2Qcemjq_kwUi06EhBmtuL7OBSFSlXmYjMTbrtEGhQeJnQqF4g3MVfRIXMPoBBaQvMf4w", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.346826, 29.735939], "type": "Point"}, "properties": {"address": "3505 Gulf Fwy, Houston, TX 77003, United States", "id": "a5933680c7c3fb0c3af444460fe49108e2f074d3", "name": "Shell", "photos": [], "place_id": "ChIJh8f2gPu-QIYRUXOjjflieG4", "rating": 3.5, "reference": "CmRRAAAA65dE4m2qveuQpHl4uM-78p8vTRbgl00zcuauUfK_TuWM6Wne7Z6FXuoN9fw_YbHr3Zgdv9JqGcxi6SWFQKXkH4iVnK2K7n2hr9TN0-u6PxQYp_ExQSck9gmbKU1aSTWiEhAMNWHUyyyOQHmjZgEagVRKGhTKuEXkDWlaijOHG0RHM7311knayw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.316063, 29.77356899999999], "type": "Point"}, "properties": {"address": "5440 I 10 East, Houston, TX 77020, United States", "id": "f22ce12f26ce5067a9befd2c0364a0d8e6081f94", "name": "Shell", "photos": [], "place_id": "ChIJdYl3oay-QIYRjMrgfHiuYeI", "rating": 4.3, "reference": "CmRSAAAAeL988M76g10aTx2rr7v_DdPVDM1eGrFHI45beL_uRVugRRQ1o1l1-vFrNeXCsGL2TvXbUjFKmWt7zaBTeIEpCa_z_G_CZudSygH6rpKM6W0B1kL4lcpA_IUjAW4XqNWREhBmYLUEhhyUH--3KI-WtufPGhT-VuRN0dXfOii6FZrOC7rNTerVbg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.18282320000002, 29.8058657], "type": "Point"}, "properties": {"address": "14760 Wallisville Rd #77049, Houston, TX 77049, United States", "id": "571890c337968a73b1da0fa79049dcd33e9b1826", "name": "Shell Gas Station", "photos": [], "place_id": "ChIJj7TLrl-kQIYRK-1PTcVreUg", "rating": "NA", "reference": "CmRRAAAAPcobSXtLFwYCpcVUmtdOt_Bq4Dw2ogz2KMPmUr0D17Zqw1OsJA8xHlmYIVCc9njVwuOM0iHDGkhN4P-mgGvqp9nbLLnNc6ZPydW-VNJoBQEcQoPu0yfJUftUktM8mXbVEhBUenZbrxVPORNAZwNk090qGhSddAadkixRvozXc9xAhK1iBXCYIw", "types": ["gas_station", "convenience_store", "food", "store", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3654938, 29.7602374], "type": "Point"}, "properties": {"address": "700 Milam St # 125, Houston, TX 77002, United States", "id": "15e18e98bcb38bed8a1e0e52a360f52e49ede007", "name": "Shell Oil Co", "photos": [], "place_id": "ChIJ-yHQKDq_QIYRELnaVCFbiQk", "rating": 3, "reference": "CmRRAAAA3oW6zCyJNy_5E1VgdXfnersqWHdpVUwODk72ih9KJ_y66c-ZYGuoPuPYtNZKQLfLozWd8sK3hkx0giVUVDCThI10WiKr4VBmAfqQa0ClH-JB7_6ND-dy13Y-hDEoFDGXEhCnK6DtKaOUfVU2p_gsG_XyGhSwh-43JulbPXvJcB18187UsBGYtA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.64317199999999, 29.690113], "type": "Point"}, "properties": {"address": "8335 S Texas 6, Houston, TX 77083, United States", "id": "30883d5d1f6bd43eab59d5f5db6ab64f23985a75", "name": "Shell", "photos": [], "place_id": "ChIJzYLoWuDdQIYRMYkyJM-lPx4", "rating": 4.2, "reference": "CmRRAAAACGlaUXr5Wtmc9tcnILYVMEG5IL6muujMH7k9a_B1DaiVVKReOHpsPOJ74TKzNJsIDq6LTubbzMv5smswiYXIDhEFyMd-Lxntqerd2Z-Vyk9SOCuTEKBpLbJEhLiZNmcdEhAkhUf_1rEfmGNCJrcXuARtGhQzb3W9Ze9_05ePcg9J3kO54UUJOw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.36766109999999, 29.7591382], "type": "Point"}, "properties": {"address": "910 Louisiana St, Houston, TX 77002, United States", "id": "0021352724df52d54019ec1367c3add6a9213250", "name": "Shell Chemicals", "photos": [], "place_id": "ChIJ-yHQKDq_QIYRpKwUxYNf1pc", "rating": 5, "reference": "CmRSAAAAKV9C-uu5XTuLFYBrDdiZsK3A0mG83onPAz9SlrgP-kU0T8do5K6sUjriieSZB2E7YsL-u2CYcGiQFV7dB34nuRKVrClRv3PaJMeC6xmUpwXnV_VTAgXrSMuf12FfhzSmEhCyGN6nvw9LQknwZ-XITnZoGhTSn8HzFV07Qf2MOdSA8gcg5CKfww", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.375708, 29.73222500000001], "type": "Point"}, "properties": {"address": "4102 Almeda RD, Houston, TX 77004, United States", "id": "38c0f74426d16401510dac6a7c11f42c0d361879", "name": "Shell", "photos": [], "place_id": "ChIJm-tuf3O_QIYRqOeliKHn5YI", "rating": 2.7, "reference": "CmRSAAAAKLYOvrewesNh5JxB7LqELZ1UUHcvJmpxgpmFJAqnxh1A3gDn3SzCYIHDugRa_W8WMr4_wX5dq3W_zJgy45JQc1Ttv4QzkpbNyJNRkJhKHnZ7TmCm1x0WOlcBW9fxJGhzEhBqkknNX9sSUhjHM_ORXHcIGhS5gRrS_6OMpP_ohTTE0cBN09SgDw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.36566099999999, 29.75684600000001], "type": "Point"}, "properties": {"address": "1000 Main St, Houston, TX 77002, United States", "id": "136bd91368a7903337a6ba59eced1feddaa5641e", "name": "Shell", "photos": [], "place_id": "ChIJgyHmO-HnQIYRqIg0YBzy5ZU", "rating": 4.8, "reference": "CmRSAAAAxw0olN7Pv14Xkj3YLMHMhocoEX1Ds0Vd3Uc3aJCYKfyFCFY24zFC8wgveRl0cSagWVhmdxKQGKW8T1MBLwd2kNBMW-dvxgunQin1bdeet85IgaUvygkGaRA1X1H1rIZeEhBEa0YBQ9ItXhXZVlecC7H9GhQn9UaI8kGCxzWiJwpM-byvUIFhMA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.33626199999999, 29.658344], "type": "Point"}, "properties": {"address": "8601 Martin Luther King Blvd, Houston, TX 77033, United States", "id": "63ab1e40f39f4a4f9443666bcbe77c2f8f46f6d4", "name": "Shell", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/116431771714044812578/photos\">rf oduol</a>"], "photo_reference": "CoQBdwAAACJypp36cjYEPnOcinRrwhszZuNBedE86cpqbG9Tx9Rp0xrHsP45EZafU2xU2JT0wvyciGbTTi0fzl7PhBm1Sh6TJI_init-7XyWoCjQO6YeWq2EM7G45bOowxkpIJhBTuNFu6PAareSmqzG8tDCFfo26Kzg_yEUcvjghp3TGLXOEhDe7cd-0F_VKrRv8i7r-q0GGhROt4UXHo4129VqrSkOz3qM5ZPg9w", "width": 5312}], "place_id": "ChIJlfRGAb6VQIYRo6gggXoJcn8", "rating": 3.9, "reference": "CmRRAAAAv7_GTj4wi6cMbz-I_cixFArNW1ZVN3Rx84j9bIXkmnBiCgrZFO0F_2ltF8IwlKbBBN7JrW4GQwfsHEirpWQcQ8g72UOEG0VUATyPZ7PwqxzEzP1LpZ_YpaaFi60HYDHyEhCbgmaoNAAVidJj9yVI1KcLGhS_PBoNCqxdd1a9V17IVp33q1wQog", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.625976, 29.759544], "type": "Point"}, "properties": {"address": "1402 Eldridge Pkwy, Houston, TX 77077, United States", "id": "534f72a95f674efa318c75ebf7fdfe31019003b5", "name": "Shell", "photos": [], "place_id": "ChIJxfpp3grcQIYRxKQZj-99VA4", "rating": 3.6, "reference": "CmRRAAAAQqAm8f8R7hwgk-3l0DWigtv4dvh4oi35kDG_LHRiucME-YohigWLLOtJ4iYtuaUu2PirLjyLAslTQKvfrjUkyWxl54T---_8Ur75XpYu2rfFYaZMywuIUPzqRY3TlKlyEhBcS78sQgLGI4CpawCCWDYcGhQX-ijbDmxpq0_peZ4WLZJwzNuuTg", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.634849, 29.902093], "type": "Point"}, "properties": {"address": "8497 Hwy 6 N, Houston, TX 77095, United States", "id": "9f5fa34f625149ee1765051a13d43a51fded8637", "name": "Shell", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/104840624709152271038/photos\">Trenton Telge</a>"], "photo_reference": "CoQBdwAAAK4w6oR2SEKgk4eguX23RgvS2Cn1o4KUndWgzp9WWn0dQZB96be0USN81_hMd9vbMIs9ZO2QWvd93GbmwvoA5tF9rIsMAEw9_ah-b_xwwbhTQAkg3BI7KubgH6B_eyoQE66dvG3WZHaUEUU9xYXAsjvwzy2ku_Q6Cyl2pbZsLgjREhBX3ZlX2j87kOd-StHD8FdnGhQhXSeTBxK00T9p-5A9yxfce_bXbQ", "width": 5312}], "place_id": "ChIJMarO2yfRQIYRvweAd7ah6lk", "rating": 3.8, "reference": "CmRRAAAAE-fZh89F3yWQULC5OspL7V5jO9fWu08cbyqOn6aCDRu9cl8VKxo7jMnhOrCr5EuaaKMPjgtGBdHsMQ5L6hjjZWLBRNg3iL1w5fDkZxPrTqvzps1YLqo4_Zyr-BJcObyhEhAkZLBcXKWZxlfwNWDa57LgGhTDZDwxP1Uq_7rfdBXVDyESNlgylw", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.565243, 29.653341], "type": "Point"}, "properties": {"address": "11503 Southwest Fwy, Houston, TX 77031, United States", "id": "eafc8e7fbe915b243dc9b151d1cc748fa092017e", "name": "Shell", "photos": [{"height": 1836, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/107934174504177841943/photos\">Jari Fuller</a>"], "photo_reference": "CoQBdwAAAARKajk0v359zfInTjgODi1ObhdqUCzYe-FmiC9fjyRkHQoZrQVz-M5jco7EHdnZkODeLGqZCKL4dQ0fuQnfpnaRbqknxZZCvr75y9lP1Pkap_i1lXisO3YQO1n9GqMNxRsVQdHpPlolrwfsGopmnxNrEve24MkpMOWz529lzuPUEhBSmX82MXKSIRsaLhAoCsucGhQ2K7hz9o4jZTpItYlZWo7MK-k_nw", "width": 3264}], "place_id": "ChIJcfq8E-HnQIYRvQ4ZJX9fTgA", "rating": 3.5, "reference": "ClRQAAAAdlzGoBuYGj-ZimumnH8zF4_OFeK_KAeH32iEqBOwJWDZvWr7oSEiEuYkpoIYYxjHey5LARzoE0DCR4sI3AwHtfqJ1nstN6Vdazdf-HxGhZ8SEIo8QJWXdAxLmxsnTkRq4JsaFPct1gT_0NgZywVVWT91EX2NYd88", "types": ["gas_station", "point_of_interest", "establishment"]}, "type": "Feature"}]

In [100]:
testArray6= [{"geometry": {"coordinates": [-95.3659038, 29.75805819999999], "type": "Point"}, "properties": {"address": "919 Milam St #2100, Houston, TX 77002, United States", "id": "7dbf977f8ebe642724c868999336f79ab8361447", "name": "Genesis Energy, LP", "photos": [], "place_id": "ChIJfy2iBDm_QIYRJ3VGXRbTSYg", "rating": 1, "reference": "CmRSAAAAzkH8cGzZOxdU70OaXzo3PElEwy5cWBYBiy6jkHtDq6NEgvyd6i-ow3achtIlKyO3YIJycZIEJbKcoAJtBmOTcGW_iS7N3AffL2JNJZnhRJmWDKDWPasgDXsI8C8HakX3EhBLhHHH3TBuGpp6ubNB2u9zGhSQ427Kc6QKdxHC23ef6BEIumno8Q", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}]

In [101]:
testArray4.extend(testArray6)


In [102]:
testArray4

[{'geometry': {'coordinates': [-95.412966, 29.706454], 'type': 'Point'},
  'properties': {'address': '2302 W Holcombe Blvd, Houston, TX 77030, United States',
   'id': '126915b3e3c7cc252e8aacbb459f3805230741b3',
   'name': 'Shell',
   'photos': [],
   'place_id': 'ChIJMw9zOmrAQIYRkpd4wCH_QlM',
   'rating': 3.5,
   'reference': 'CmRRAAAAHWcQxO3UYvnEPP_LpHTHN09gPwRwZuS78sypQY-HxSwqmcIlKjExzrHiKTTGb9ARAl1aQF9QKSgRdHf1FkKjjhmgiXIZgT6YZOdB4ZUcDrOD30yaHXyFidqx0ab2ycbTEhAzbEAg3CmeN7v2eZ-RVKtnGhSWBxHKgily1adbesV1S4vEV3zisg',
   'types': ['gas_station', 'point_of_interest', 'establishment']},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-95.294772, 29.702266], 'type': 'Point'},
  'properties': {'address': '7001 Gulf Fwy, Houston, TX 77087, United States',
   'id': '9364f2933512ea2c8c79073edf93948b93c6eb0a',
   'name': 'Shell',
   'photos': [{'height': 2988,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108597419151098384564/photos">Joshua Suaris</a>'],
     '